# My Capstone Project

*Before we get the data and start exploring it, let's download all the dependencies that we will need*

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [12]:
# Tarragona coordinate

latitude = 41.123312
longitude = 1.248521


## Create a map of Tarragona with neighbordhoods superimposed on top

In [35]:
map_Tarragona = folium.Map(location=[latitude, longitude], zoom_start=13)


In [36]:
map_Tarragona

In [37]:
import pandas as pd 
import os

In [38]:
pwd

'/resources/labs/Capstoe'

In [39]:
path = '/resources/labs/Capstoe/tarragona.csv'

df = pd.read_csv(path)

In [40]:
df.head()

,ciudad,barrio,latitud,longitud
0,Tarragona,Sant Pere i Sant Pau,41.137686,1.252032
1,Tarragona,Torreforta,41.118950,1.216849
2,Tarragona,Salou,41.080084,1.132025
3,Tarragona,La Pineda,41.079023,1.182240
4,Tarragona,Vila Seca,41.102044,1.146068


In [44]:
df.columns

Index(['ciudad', ' barrio', ' latitud', ' longitud'], dtype='object')

In [45]:
for lat, lng, ciudad, barrio in zip(df[' latitud'], df[' longitud'], df[' barrio'], df['ciudad']):
    label = '{}, {}'.format(barrio, ciudad)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng], 
        radius=6,
        popup=label,
        color='red',
        fill=True, 
        fill_color='yellow', 
        fill_opacity=1.0, 
        parse_html=False).add_to(map_Tarragona)
    
map_Tarragona
    

# Define Foursquare Credentials and Version

In [47]:
CLIENT_ID = 'Y0P0PGSBLUDSIXYQ2451UJ2KBTVKHVRI4TSSFARE2PAO2BYP' # your Foursquare ID
CLIENT_SECRET = 'K5HAL5RSY340WZ3VPMUYGPPTSQ2SJBCOYQBTNFRLSRB4K4L5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y0P0PGSBLUDSIXYQ2451UJ2KBTVKHVRI4TSSFARE2PAO2BYP
CLIENT_SECRET:K5HAL5RSY340WZ3VPMUYGPPTSQ2SJBCOYQBTNFRLSRB4K4L5


In [49]:
tarragona_data = df

In [59]:
tarragona_data.loc[2, ' barrio']

' Salou'

In [60]:
barrio_latitude = tarragona_data.loc[2, ' latitud']
barrio_longitude = tarragona_data.loc[2, ' longitud']
barrio_name = tarragona_data.loc[2, ' barrio']

print('Latitude and longitude values of {} are {}, {}.'.format(barrio_name, 
                                                              barrio_latitude, 
                                                              barrio_longitude))

Latitude and longitude values of  Salou are 41.080084, 1.132025.


In [61]:
Limit = 100
radius= 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    barrio_latitude, 
    barrio_longitude, 
    radius, 
    Limit)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=Y0P0PGSBLUDSIXYQ2451UJ2KBTVKHVRI4TSSFARE2PAO2BYP&client_secret=K5HAL5RSY340WZ3VPMUYGPPTSQ2SJBCOYQBTNFRLSRB4K4L5&v=20180605&ll=41.080084,1.132025&radius=500&limit=100'

In [62]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5819dec53093002cefe85d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Vilafortuny',
  'headerFullLocation': 'Vilafortuny',
  'headerLocationGranularity': 'city',
  'totalResults': 32,
  'suggestedBounds': {'ne': {'lat': 41.0845840045, 'lng': 1.1379836774188945},
   'sw': {'lat': 41.0755839955, 'lng': 1.1260663225811056}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e0f7aaffa76d62f4452e6c0',
       'name': 'La Morera de Pablo & Ester',
       'location': {'address': 'C. Berenguer de Palou, 10',
        'lat': 41.0804859000759,
        'lng': 1.1320965661238425,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.0804859

In [63]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [65]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,La Morera de Pablo & Ester,Mediterranean Restaurant,41.080486,1.132097
1,La Ibense Salou Heladeria Palas,Ice Cream Shop,41.080603,1.131083
2,Meson del Segoviano,Spanish Restaurant,41.079193,1.130446
3,Sunset St.,Tapas Restaurant,41.080196,1.131168
4,Mercat Municipal de Salou,Food & Drink Shop,41.078893,1.132206


In [70]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

32 venues were returned by Foursquare.


In [71]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Barrio', 
                  'Barrio Latitude', 
                  'Barrio Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
nearby_venues.shape

(32, 4)

In [74]:
tarragona_venues = getNearbyVenues(names=tarragona_data[' barrio'], 
                                  latitudes=tarragona_data[' latitud'], 
                                  longitudes=tarragona_data[' longitud']
                                  )

 Sant Pere i Sant Pau
 Torreforta
 Salou
 La Pineda
 Vila Seca
 Bonavista


In [75]:
tarragona_venues.shape

(71, 7)

In [85]:
tarragona_venues.columns

Index(['Barrio', 'Barrio Latitude', 'Barrio Longitude', 'Venue',
       'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [86]:
tarragona_venues.groupby('Barrio').count()

,Barrio Latitude,Barrio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Barrio,,,,,,
Bonavista,5,5,5,5,5,5
La Pineda,21,21,21,21,21,21
Salou,32,32,32,32,32,32
Sant Pere i Sant Pau,4,4,4,4,4,4
Torreforta,6,6,6,6,6,6
Vila Seca,3,3,3,3,3,3


In [87]:
print('There are {} uniques categories.'.format(len(tarragona_venues['Venue Category'].unique())))

There are 37 uniques categories.


# Analyze Each Neighborhood

In [88]:
# one hot encoding
tarragona_onehot = pd.get_dummies(tarragona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tarragona_onehot['Barrio'] = tarragona_venues['Barrio'] 

# move neighborhood column to the first column
fixed_columns = [tarragona_onehot.columns[-1]] + list(tarragona_onehot.columns[:-1])
tarragona_onehot = tarragona_onehot[fixed_columns]

tarragona_onehot.head()

,Barrio,Art Museum,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Brewery,Café,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Diner,Donut Shop,Farmers Market,Fish & Chips Shop,Food & Drink Shop,Gastropub,Grocery Store,Gym,Hotel,Ice Cream Shop,Italian Restaurant,Liquor Store,Mediterranean Restaurant,Modern European Restaurant,Pizza Place,Pool,Restaurant,Seafood Restaurant,Soccer Field,Spanish Restaurant,Stadium,Supermarket,Tapas Restaurant,Train Station
0,Sant Pere i Sant Pau,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,Sant Pere i Sant Pau,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sant Pere i Sant Pau,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Sant Pere i Sant Pau,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Torreforta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [89]:
tarragona_onehot.shape

(71, 38)

In [90]:
tarragona_grouped = tarragona_onehot.groupby('Barrio').mean().reset_index()
tarragona_grouped

,Barrio,Art Museum,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beer Garden,Brewery,Café,Cocktail Bar,Coffee Shop,Concert Hall,Department Store,Diner,Donut Shop,Farmers Market,Fish & Chips Shop,Food & Drink Shop,Gastropub,Grocery Store,Gym,Hotel,Ice Cream Shop,Italian Restaurant,Liquor Store,Mediterranean Restaurant,Modern European Restaurant,Pizza Place,Pool,Restaurant,Seafood Restaurant,Soccer Field,Spanish Restaurant,Stadium,Supermarket,Tapas Restaurant,Train Station
0,Bonavista,0.00000,0.00,0.200000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0000,0.2,0.000000,0.00000,0.000000,0.00000,0.2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.2000,0.000000,0.000000,0.200000,0.00000
1,La Pineda,0.00000,0.00,0.047619,0.047619,0.00000,0.047619,0.00000,0.00,0.095238,0.047619,0.00000,0.00000,0.000000,0.047619,0.0000,0.0,0.047619,0.00000,0.047619,0.00000,0.0,0.095238,0.142857,0.047619,0.00000,0.047619,0.047619,0.0000,0.000000,0.095238,0.047619,0.00,0.0000,0.000000,0.000000,0.047619,0.00000
2,Salou,0.03125,0.00,0.000000,0.000000,0.03125,0.000000,0.03125,0.00,0.062500,0.031250,0.03125,0.03125,0.000000,0.000000,0.0625,0.0,0.000000,0.03125,0.031250,0.03125,0.0,0.156250,0.031250,0.000000,0.03125,0.093750,0.000000,0.0625,0.000000,0.062500,0.000000,0.00,0.0625,0.000000,0.000000,0.062500,0.03125
3,Sant Pere i Sant Pau,0.00000,0.25,0.000000,0.000000,0.00000,0.000000,0.00000,0.25,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0000,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.25,0.2500,0.000000,0.000000,0.000000,0.00000
4,Torreforta,0.00000,0.00,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.00000,0.166667,0.000000,0.0000,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.166667,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.166667,0.000000,0.00,0.0000,0.166667,0.333333,0.000000,0.00000
5,Vila Seca,0.00000,0.00,0.000000,0.333333,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.0000,0.0,0.000000,0.00000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.333333,0.333333,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00000


In [91]:
tarragona_grouped.shape

(6, 38)

In [92]:
num_top_venues = 5

for hood in tarragona_grouped['Barrio']:
    print("----"+hood+"----")
    temp = tarragona_grouped[tarragona_grouped['Barrio'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Bonavista----
                venue  freq
0                 Bar   0.2
1    Tapas Restaurant   0.2
2  Spanish Restaurant   0.2
3      Farmers Market   0.2
4                 Gym   0.2


---- La Pineda----
                venue  freq
0      Ice Cream Shop  0.14
1               Hotel  0.10
2          Restaurant  0.10
3                Café  0.10
4  Italian Restaurant  0.05


---- Salou----
                      venue  freq
0                     Hotel  0.16
1  Mediterranean Restaurant  0.09
2                Donut Shop  0.06
3          Tapas Restaurant  0.06
4        Spanish Restaurant  0.06


---- Sant Pere i Sant Pau----
                venue  freq
0              Bakery  0.25
1  Spanish Restaurant  0.25
2        Soccer Field  0.25
3             Brewery  0.25
4         Pizza Place  0.00


---- Torreforta----
              venue  freq
0       Supermarket  0.33
1             Hotel  0.17
2           Stadium  0.17
3        Restaurant  0.17
4  Department Store  0.17


---- Vila Seca----
    

In [93]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Barrio']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Barrio'] = tarragona_grouped['Barrio']

for ind in np.arange(tarragona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tarragona_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bonavista,Farmers Market,Tapas Restaurant,Bar,Spanish Restaurant,Gym,Cocktail Bar,Donut Shop,Diner,Department Store,Concert Hall
1,La Pineda,Ice Cream Shop,Café,Hotel,Restaurant,Gastropub,Diner,Fish & Chips Shop,Tapas Restaurant,Cocktail Bar,Italian Restaurant
2,Salou,Hotel,Mediterranean Restaurant,Spanish Restaurant,Café,Restaurant,Pizza Place,Donut Shop,Tapas Restaurant,Train Station,Grocery Store
3,Sant Pere i Sant Pau,Bakery,Spanish Restaurant,Soccer Field,Brewery,Train Station,Cocktail Bar,Donut Shop,Diner,Department Store,Concert Hall
4,Torreforta,Supermarket,Stadium,Department Store,Restaurant,Hotel,Train Station,Café,Diner,Concert Hall,Coffee Shop


# Cluster Neighborhoods

In [109]:
kclusters = 5

tarragona_grouped_clustering = tarragona_grouped.drop('Barrio', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tarragona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 2, 1, 0, 4], dtype=int32)

In [113]:
neighborhoods_venues_sorted.columns

Index(['Cluster Labels', 'Barrio', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

In [118]:
tarragona_merged = tarragona_data

tarragona_merged = tarragona_data.join(neighborhoods_venues_sorted.set_index('Barrio'), on=' barrio')


In [119]:
tarragona_merged.head()

,ciudad,barrio,latitud,longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tarragona,Sant Pere i Sant Pau,41.137686,1.252032,1,Bakery,Spanish Restaurant,Soccer Field,Brewery,Train Station,Cocktail Bar,Donut Shop,Diner,Department Store,Concert Hall
1,Tarragona,Torreforta,41.118950,1.216849,0,Supermarket,Stadium,Department Store,Restaurant,Hotel,Train Station,Café,Diner,Concert Hall,Coffee Shop
2,Tarragona,Salou,41.080084,1.132025,2,Hotel,Mediterranean Restaurant,Spanish Restaurant,Café,Restaurant,Pizza Place,Donut Shop,Tapas Restaurant,Train Station,Grocery Store
3,Tarragona,La Pineda,41.079023,1.182240,2,Ice Cream Shop,Café,Hotel,Restaurant,Gastropub,Diner,Fish & Chips Shop,Tapas Restaurant,Cocktail Bar,Italian Restaurant
4,Tarragona,Vila Seca,41.102044,1.146068,4,Basketball Court,Restaurant,Pool,Train Station,Café,Donut Shop,Diner,Department Store,Concert Hall,Coffee Shop


In [121]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tarragona_merged[' latitud'], tarragona_merged[' longitud'], tarragona_merged[' barrio'], tarragona_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# EXAMINE CLUSTERS

## Cluster 1

In [122]:
tarragona_merged.loc[tarragona_merged['Cluster Labels'] == 0, tarragona_merged.columns[[1] + list(range(5, tarragona_merged.shape[1]))]]

,barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Torreforta,Supermarket,Stadium,Department Store,Restaurant,Hotel,Train Station,Café,Diner,Concert Hall,Coffee Shop


## Cluster 2

In [123]:
tarragona_merged.loc[tarragona_merged['Cluster Labels'] == 1, tarragona_merged.columns[[1] + list(range(5, tarragona_merged.shape[1]))]]

,barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sant Pere i Sant Pau,Bakery,Spanish Restaurant,Soccer Field,Brewery,Train Station,Cocktail Bar,Donut Shop,Diner,Department Store,Concert Hall


## Cluster 3

In [126]:
tarragona_merged.loc[tarragona_merged['Cluster Labels'] == 2, tarragona_merged.columns[[1] + list(range(5, tarragona_merged.shape[1]))]]

,barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Salou,Hotel,Mediterranean Restaurant,Spanish Restaurant,Café,Restaurant,Pizza Place,Donut Shop,Tapas Restaurant,Train Station,Grocery Store
3,La Pineda,Ice Cream Shop,Café,Hotel,Restaurant,Gastropub,Diner,Fish & Chips Shop,Tapas Restaurant,Cocktail Bar,Italian Restaurant


## Cluster 4

In [128]:
tarragona_merged.loc[tarragona_merged['Cluster Labels'] == 3, tarragona_merged.columns[[1] + list(range(5, tarragona_merged.shape[1]))]]

,barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Bonavista,Farmers Market,Tapas Restaurant,Bar,Spanish Restaurant,Gym,Cocktail Bar,Donut Shop,Diner,Department Store,Concert Hall


## Cluster 5

In [129]:
tarragona_merged.loc[tarragona_merged['Cluster Labels'] == 4, tarragona_merged.columns[[1] + list(range(5, tarragona_merged.shape[1]))]]

,barrio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Vila Seca,Basketball Court,Restaurant,Pool,Train Station,Café,Donut Shop,Diner,Department Store,Concert Hall,Coffee Shop
